In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [41]:
mnist = input_data.read_data_sets("./data/mnist/",one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz


W0207 09:07:39.058389 139679559546688 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


In [7]:
a, b =mnist.train.next_batch(11)
print(a.dtype, b.dtype)


float32 uint8


In [34]:
# 自定义的卷积模型
# 1. 准备数据的占位符 x [None, 784], y_true [None, 10]
with tf.variable_scope("data"):
    x = tf.placeholder(tf.float32, [None, 784])
    y_true = tf.placeholder(tf.int8, [None, 10])
    
# 2. 一卷积层 卷积5*5*1, 32个， strides=1, 激活tf.nn.relu 池化
with tf.variable_scope("conv1"):
    # 随机初始化卷积Filter权重 [5, 5, 1, 32]，偏置 [32]
    w_conv1 = tf.Variable(tf.random_normal([5,5,1,32],mean=0.0,stddev=1.0),
                         name="w_conv1")
    b_conv1 = tf.Variable(tf.constant(0.0, shape=[32]),name="conv1_b")

    # 对x进行形状改变[None 784] [None, 28, 28, 1]
    x_reshape = tf.reshape(x, [-1, 28, 28, 1])
    
    # [None,28,28,1] --> [None,28,28,32]
    x_relu1 = tf.nn.relu(tf.nn.conv2d(x_reshape, w_conv1, strides=[1,1,1,1], padding="SAME")+b_conv1)
    
    # 池化窗口=2*2, strides=2 [None, 28, 28, 32] --> [None, 14, 14, 32]
    x_pool1 = tf.nn.max_pool(x_relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    
# 3. 二卷积层卷积：Filter=5*5*32, 64个filter，strides=1
with tf.variable_scope("conv2"):
    # 随机初始化卷积Filter W [5, 5, 32, 64] bias=[64]
    w_conv2 = tf.Variable(tf.random_normal([5,5,32,64],mean=0.0,stddev=1.0))
    b_conv2 = tf.Variable(tf.constant(0.0, shape=[64]))
    
    # [None 14,14,32] ==> [None,14,14,64]
    x_relu2 = tf.nn.relu(tf.nn.conv2d(x_pool1,w_conv2,strides=[1,1,1,1],padding="SAME")+b_conv2)
    
    # 池化窗口：2*2 strides:2 [None, 14, 14,64] --> [None,7,7,64]
    x_pool2 = tf.nn.max_pool(x_relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    
# 4. 全连接层 [None, 7, 7, 64]  -->
# [None, 7*7*64]*[7*7*64, 10]+[10] = [None, 10]
with tf.variable_scope("fc"):
    w_fc = tf.Variable(tf.random_normal([7*7*64, 10],mean=0.0,stddev=1.0))
    b_fc = tf.Variable(tf.constant(0.0,shape=[10]))

    #修改x 形状 [None,7,7,64] --》[None, 7*7*64]
    x_fc_reshape = tf.reshape(x_pool2, [-1,7*7*64])

    # 进行矩阵运算，得出每个样本的10个结果
    y_predict = tf.matmul(x_fc_reshape, w_fc) + b_fc
#     print(y_predict)

In [35]:
# 5. 进行交叉熵损失计算
# 求出所有样本的损失，然后求平均值
with tf.variable_scope("soft_cross"):
    fc_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_predict)
#     print("fc_entropy:", fc_entropy)
    
    loss = tf.reduce_mean(fc_entropy)
#     print(loss)

# 6. 梯度下降求出损失
with tf.variable_scope("optimizer"):
    train_op = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)
    
# 7. 计算准确率
with tf.variable_scope("acc"):
    equal_list = tf.equal(tf.argmax(y_true,1),tf.arg_max(y_predict,1))
    accuracy = tf.reduce_mean(tf.cast(equal_list, tf.float32))
#     print(accuracy)
        

In [60]:
#定义一个初始化变量OP
init_op = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init_op)
    start_time = time.time()
    # 循环训练
    for i in range(10000):
        mnist_x, mnist_y = mnist.train.next_batch(50)
        sess.run(train_op,feed_dict={x:mnist_x,y_true:mnist_y})
        
        if i %100 == 0:
            print("%d step,acc:%f" %(i, sess.run(accuracy, feed_dict={x:mnist_x,y_true:mnist_y})))
    
    print(time.time()-start_time())
            

0 step,acc:0.180000
100 step,acc:0.400000
200 step,acc:0.460000
300 step,acc:0.500000
400 step,acc:0.740000
500 step,acc:0.660000
600 step,acc:0.700000
700 step,acc:0.600000
800 step,acc:0.700000
900 step,acc:0.780000
1000 step,acc:0.760000
1100 step,acc:0.780000
1200 step,acc:0.820000
1300 step,acc:0.860000
1400 step,acc:0.880000
1500 step,acc:0.780000
1600 step,acc:0.820000
1700 step,acc:0.800000
1800 step,acc:0.800000
1900 step,acc:0.780000
2000 step,acc:0.880000
2100 step,acc:0.840000
2200 step,acc:0.820000
2300 step,acc:0.720000
2400 step,acc:0.880000
2500 step,acc:0.960000
2600 step,acc:0.780000
2700 step,acc:0.840000
2800 step,acc:0.860000
2900 step,acc:0.780000
3000 step,acc:0.820000
3100 step,acc:0.820000
3200 step,acc:0.940000
3300 step,acc:0.820000
3400 step,acc:0.820000
3500 step,acc:0.880000
3600 step,acc:0.860000
3700 step,acc:0.820000
3800 step,acc:0.880000
3900 step,acc:0.860000
4000 step,acc:0.860000
4100 step,acc:0.860000
4200 step,acc:0.800000
4300 step,acc:0.860000


TypeError: 'float' object is not callable

In [17]:

help(tf.nn.softmax_cross_entropy_with_logits)

Help on function softmax_cross_entropy_with_logits in module tensorflow.python.ops.nn_ops:

softmax_cross_entropy_with_logits(_sentinel=None, labels=None, logits=None, dim=-1, name=None, axis=None)
    Computes softmax cross entropy between `logits` and `labels`. (deprecated)
    
    Instructions for updating:
    
    Future major versions of TensorFlow will allow gradients to flow
    into the labels input on backprop by default.
    
    See `tf.nn.softmax_cross_entropy_with_logits_v2`.
    
    
    Measures the probability error in discrete classification tasks in which the
    classes are mutually exclusive (each entry is in exactly one class).  For
    example, each CIFAR-10 image is labeled with one and only one label: an image
    can be a dog or a truck, but not both.
    
    **NOTE:**  While the classes are mutually exclusive, their probabilities
    need not be.  All that is required is that each row of `labels` is
    a valid probability distribution.  If they are not, t

In [59]:
import time
a = time.time()

time.sleep(2)
print(time.time() -a)

2.0023791790008545


Help on function max_pool in module tensorflow.python.ops.nn_ops:

max_pool(value, ksize, strides, padding, data_format='NHWC', name=None, input=None)
    Performs the max pooling on the input.
    
    Args:
      value: A 4-D `Tensor` of the format specified by `data_format`.
      ksize: An int or list of `ints` that has length `1`, `2` or `4`.
        The size of the window for each dimension of the input tensor.
      strides: An int or list of `ints` that has length `1`, `2` or `4`.
        The stride of the sliding window for each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm.
        See the "returns" section of `tf.nn.convolution` for details.
      data_format: A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.
      name: Optional name for the operation.
      input: Alias for value.
    
    Returns:
      A `Tensor` of format specified by `data_format`.
      The max pooled output tensor.

